This pipeline load the data in duckdb -- This work

In [19]:
import dlt
from rest_api import rest_api_source

source = rest_api_source({
        "client": {
            "base_url": "https://api.spacexdata.com/v4"
        },

        "resources": [
            {
                "name": "launches",
                "write_disposition": "merge",
                "primary_key": "id",
                "endpoint": {
                    "path": "launches",
                },
            },
            {
                "name": "rockets",
                "endpoint": {
                    "path": "rockets",
                },
            },
            {
                "name": "crew",
                "endpoint": {
                    "path": "crew",
                },
            },
            {
                "name": "payloads",
                "endpoint": {
                    "path": "payloads",
                },
            }
        ]
    })

pipeline = dlt.pipeline(
    pipeline_name="rest_api_example",
    destination="duckdb",
    dataset_name="rest_api_data0",
)

load_info = pipeline.run(source)
print(load_info)

2024-09-03 17:43:07,723|[WARNING]|55801|8497359168|dlt|client.py|detect_paginator:292|Fallback paginator used: SinglePagePaginator at 147b60b80. Please provide right paginator manually.
2024-09-03 17:43:07,900|[WARNING]|55801|8497359168|dlt|client.py|detect_paginator:292|Fallback paginator used: SinglePagePaginator at 147c9dbd0. Please provide right paginator manually.
2024-09-03 17:43:08,085|[WARNING]|55801|8497359168|dlt|client.py|detect_paginator:292|Fallback paginator used: SinglePagePaginator at 133ae6200. Please provide right paginator manually.
2024-09-03 17:43:08,360|[WARNING]|55801|8497359168|dlt|client.py|detect_paginator:292|Fallback paginator used: SinglePagePaginator at 147dc4820. Please provide right paginator manually.


Pipeline rest_api_example load step completed in 2.11 seconds
1 load package(s) were loaded to destination duckdb and into dataset rest_api_data0
The duckdb destination used duckdb:////Users/alejandrogonzalezbueno/Projects/dlt_workshop/rest_api_example.duckdb location to store data
Load package 1725378187.1712892 is LOADED and contains no failed jobs


In [ ]:
import duckdb
import pandas as pd

# Conectar a la base de datos
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")

# Ejecutar la consulta y obtener el DataFrame
stats_table = conn.sql('show tables').df()

display(stats_table)

In [ ]:
import dlt
from rest_api import rest_api_source

source = rest_api_source({
        "client": {
            "base_url": "https://api.spacexdata.com/v4"
        },

        "resources": [
            {
                "name": "launches",
                "write_disposition": "append",
                "endpoint": {
                    "path": "launches",
                },
            },
            {
                "name": "payloads",
                "endpoint": {
                    "path": "payloads/{payload_id}",
                    "params": {
                        "payload_id": {
                            "type": "resolve",
                            "resource": "launches",
                            "field": "payloads",
                            "data_selector": "payloads" 
                        }
                    }
                },
                "include_from_parent": ["id"],
            }
        ]
    })

# Fetch both launches and payloads in the source
data = source.with_resources("launches", "payloads")

pipeline = dlt.pipeline(
    pipeline_name="rest_api_example_08",
    destination="duckdb",
    dataset_name="rest_api_data",
)

# Run the pipeline
load_info = pipeline.run(source)
print(load_info)

In [ ]:
import duckdb
import pandas as pd

# Conectar a la base de datos
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")

# Ejecutar la consulta y obtener el DataFrame
stats_table = conn.sql('show tables').df()

display(stats_table)

In [ ]:
import dlt
from rest_api import rest_api_source

source = rest_api_source({
        "client": {
            "base_url": "https://api.spacexdata.com/v4"
        },

        "resources": [
            {
                "name": "payloads",
                "write_disposition": "replace",
                "endpoint": {
                    "path": "payloads",
                },
            },
            {
                "name": "launches",
                "endpoint": {
                    "path": "launches/{launch_id}",  # singular payload_id
                    "params": {
                        "launch_id": {
                            "type": "resolve",    # Use 'resolve' to handle individual IDs
                            "resource": "payloads",
                            "field": "launch",
                        }
                    }
                },
                "include_from_parent": ["id"],
            }
        ]
    })

# Fetch both launches and payloads in the source
data = source.with_resources("launches", "payloads")

pipeline = dlt.pipeline(
    pipeline_name="rest_api_example_08",
    destination="duckdb",
    dataset_name="rest_api_data",
)

# Run the pipeline
load_info = pipeline.run(source)
print(load_info)